## Importing The Necessary Libraries

In [1]:
import sagemaker
import boto3
from sagemaker.amazon.amazon_estimator import get_image_uri 
from sagemaker.session import s3_input, Session

## Creating an S3 Bucket

In [2]:
bucket_name = 'loanapplication1'
my_region = boto3.session.Session().region_name
print(my_region)

us-east-1


In [3]:
s3 = boto3.resource('s3')
try:
    if  my_region == 'us-east-1':
        s3.create_bucket(Bucket=bucket_name)
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ',e)

S3 bucket created successfully


In [4]:
# setting an output path where the trained model will be saved
prefix = 'xgboost-as-a-built-in-algo'
output_path ='s3://{}/{}/output'.format(bucket_name, prefix) # 's3://loanaplication1/xgboost-as-a-built-in-algo//output'
print(output_path)


s3://loanapplication1/xgboost-as-a-built-in-algo/output


## Data Importation

In [5]:
import pandas as pd
import urllib

In [6]:
url = 'https://github.com/chetanhalai3c/Loan-Prediction/blob/main/Loan-processed_data.xlsx'

In [7]:
df = pd.read_csv('https://s3.console.aws.amazon.com/s3/object/loanapplication1?region=us-east-1&prefix=Loan-processed_data.csv')

In [8]:
for bucket in s3.buckets.all():
    print(bucket)

s3.Bucket(name='loanapplication1')
s3.Bucket(name='sagemaker-studio-a5y6sg2zrh')
s3.Bucket(name='sagemaker-us-east-1-461558334125')


In [9]:
for file in bucket.objects.all():
    print(file.key)

In [10]:
#s3://loanapplication1/loanData/ # https://loanapplication1.s3.amazonaws.com/loanData/Loan-processed_data.csv

In [11]:
import urllib
try:
    urllib.request.urlretrieve ("https://loanapplication1.s3.amazonaws.com/loanData/Loan-processed_data.csv", "loan_clean_data.csv")
    print('Success: download completed.')
except Exception as e:
    print('Data load error: ',e)

try:
    model_data = pd.read_csv('./loan_clean_data.csv')
    print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

Data load error:  HTTP Error 403: Forbidden
Data load error:  [Errno 2] No such file or directory: './loan_clean_data.csv'


In [12]:
# Getting data from AWS S3 bucket
s3 = boto3.client('s3')
obj = s3.get_object(Bucket = 'loanapplication1',Key = 'loanData/Loan-processed_data.csv')

df = pd.read_csv(obj['Body'])

In [13]:
df.head()

,Loan Status,Current Loan Amount,Credit Score,Annual Income,Monthly Debt,Years of Credit History,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,...,Years in current job_one or less year,Years in current job_six or seven years,Years in current job_two or three years,Home Ownership_Own Home,Home Ownership_Rent,Purpose_business,Purpose_essentials,Purpose_home_improvement,Purpose_luxury_purchase,Purpose_others
0,0,12232,7280,46643.0,777,18.0,12,0,6762,7946,...,1,0,0,0,1,0,0,0,0,0
1,0,25014,7330,81099.0,892,26.7,14,0,35706,77961,...,0,0,0,0,0,0,0,0,0,0
2,0,16117,7240,60438.0,1244,16.7,11,1,11275,14815,...,0,0,0,0,0,0,0,1,0,0
3,0,11716,7400,34171.0,990,10.0,21,0,7009,43533,...,0,0,1,0,1,0,0,0,0,0
4,0,9789,6860,47003.0,503,16.7,13,1,16913,19553,...,0,0,0,0,0,0,0,1,0,0


In [14]:
# df = pd.read_csv('s3://loanapplication1/loanData/Loan-processed_data.csv') 

In [15]:
# try:
#     urllib.request.urlretrieve (url)
#     print('Success: downloaded processed data')
# except Exception as e:
#     print('Data load error: ',e)

# try:
#     model_data = pd.read_excel( )
#     print('Success: Data loaded into dataframe.')
# except Exception as e:
#     print('Data load error: ',e)

#this function was supposed to import the data from the github link - however we opted to transform the data to csv 
# and store the data into the s3 bucket and then call it - (see above)

## Spliting data into trainset and test set 

In [16]:
import numpy as np
train_data, test_data = np.split(df.sample(frac=1, random_state=42), [int(0.7 * len(df))])
print(train_data.shape, test_data.shape)

(56414, 25) (24178, 25)


In [17]:
df['Loan Status'].unique()

array([0, 1])

In [18]:
import os
pd.concat([train_data['Loan Status'], train_data.drop('Loan Status', 
                                                axis=1)], 
                                                axis=1).to_csv('train.csv', index=False, header=False)


boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
s3_input_train = sagemaker.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')

In [19]:
# Test Data Into Buckets
pd.concat([test_data['Loan Status'], test_data.drop('Loan Status', axis=1)], axis=1)\
.to_csv('test.csv', index=False, header=False)
           
                 
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')
s3_input_test = sagemaker.TrainingInput(s3_data='s3://{}/{}/test'.format(bucket_name, prefix), content_type='csv')

## Buillding Model - XGboost

In [20]:
container = get_image_uri(boto3.Session().region_name,
                          'xgboost', 
                          repo_version='1.0-1')


The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [21]:
# initialize hyperparameters
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"binary:logistic",
        "num_round":50
        }

In [22]:
estimator = sagemaker.estimator.Estimator(image_uri=container, 
                                          hyperparameters=hyperparameters,
                                          role=sagemaker.get_execution_role(),
                                          train_instance_count=1, 
                                          train_instance_type='ml.m5.2xlarge', 
                                          train_volume_size=5,
                                          output_path=output_path,
                                          train_use_spot_instances=True,
                                          train_max_run=300,
                                          train_max_wait=600)

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_max_run has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_use_spot_instances has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_max_wait has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_volume_size has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
NOTEBOOK_METADATA_FILE detected but failed to get valid domain and user from it.


In [23]:
estimator.fit({'train': s3_input_train,'validation': s3_input_test})

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2023-09-27-16-10-34-861


2023-09-27 16:10:35 Starting - Starting the training job...
2023-09-27 16:10:50 Starting - Preparing the instances for training......
2023-09-27 16:12:02 Downloading - Downloading input data......
2023-09-27 16:12:53 Training - Training image download completed. Training in progress..[2023-09-27 16:13:03.475 ip-10-0-208-158.ec2.internal:7 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Single node training.
[16

## Deploying Machine Learning Model End Point

In [24]:
xgb_predictor = estimator.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2023-09-27-16-13-47-461
INFO:sagemaker:Creating endpoint-config with name sagemaker-xgboost-2023-09-27-16-13-47-461
INFO:sagemaker:Creating endpoint with name sagemaker-xgboost-2023-09-27-16-13-47-461


-------!

In [25]:
from sagemaker.serializers import CSVSerializer

In [31]:
test_data_array = test_data.drop('Loan Status', axis=1).values
#xgb_predictor.content_type = 'csv' 
xgb_predictor.serializer = CSVSerializer('text/csv')
predictions = xgb_predictor.predict(test_data_array).decode('utf-8') 
predictions_array = np.fromstring(predictions[1:], sep=',') # turn the prediction into an array
print(predictions_array.shape)


(24178,)


In [32]:
predictions_array

array([0.83247101, 0.72646326, 0.74559385, ..., 0.62559789, 0.00151734,
       0.77625239])

In [36]:
cm = pd.crosstab(index=test_data['Loan Status'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "Given", "Not Given"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("Given", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Not Given", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))


Overall Classification Rate: 76.8%

Predicted      Given          Not Given
Observed
Given          85% (2314)    24% (5198)
Not Given       15% (419)     76% (16247) 



**Observatio**

- 85% of the time - the model was able to predict that the loan was 'Given' and that was the case
- 24% of the time - the model miss classified 'Not Given' as 'Given' 
- 15% of the time - the model miss classificed 'Given' as 'Not Given"
- 76% of the time - the model stated 'Not Given' and that was the case 


## Deleting the Resources after use 

- The code below automatically deletes provisioned resouces to avoid encuring extra cost.

In [27]:
# sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)
# bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
# bucket_to_delete.objects.all().delete()